# 1.0 Um problema de classificação de ponta-a-ponta usando NLP

## 1.1 Descrição do dataset

O dataset contém informações gerais de 5.000 processos julgados
nos Juizados Especiais Federais dentro das Seções Judiciárias do Tribunal Regional Federal da 5a Região (https://www.trf5.jus.br). Os dados são oriundos da raspagem da consulta pública processual. Além disso, ele possui 46 colunas, das quais duas possuem texto livre:
"conteudo_sentenca" e "conteudo_acordao".

O dataset pode ser baixado no link a seguir: https://jacob.al/dataset_juizados 

Ao longo dos notebooks, vão ser realizados os seguintes passos:

1. Importação do dataset **(concluído)**
2. Análise exploratória dos dados **(concluído)**
3. Pré-processamento
4. Verificação dos dados
5. Segregação dos dados
6. Treinamento
7. Teste

<center><img width="600" src="https://drive.google.com/uc?export=view&id=1fKGuR5U5ECf7On6Zo1UWzAIWZrMmZnGc"></center>


## 1.2 Instalação e importação das bibliotecas

In [ ]:
!pip install wandb

In [ ]:
import wandb
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 1.3 Pré-processamento

### 1.3.1 Login no wandb


In [ ]:
# Login to Weights & Biases
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### 1.3.2 Artefatos

In [ ]:
input_artifact="nlp_bolsa/raw_data.csv:latest"
artifact_name="preprocessed_data.csv"
artifact_type="clean_data"
artifact_description="Data after preprocessing"

### 1.3.3 Configurando o Wandb e limpando o dataset

Depois do passo de coleta dos dados, o artefato *raw_data* foi gerado. Agora, após o pré-processamento, vamos gerar um novo artefato chamado *clean_data*.

Um ponto importante é que a modelagem vai se basear no conteúdo da coluna `conteudo_sentenca` para tentar prever o assunto processual da coluna `assunto_cnj`. Logo, vamos deixar apenas a coluna textual como feature e a coluna do assunto como variável alvo.

In [ ]:
# Criação de um novo job_type
run = wandb.init(project="nlp_bolsa", job_type="process_data")

wandb: Currently logged in as: morsinaldo. Use `wandb login --relogin` to force relogin


In [ ]:
# Download da última versão do artefato raw_data.csv
artifact = run.use_artifact(input_artifact)

# Criação de um dataframe do artefato
df = pd.read_csv(artifact.file())

In [ ]:
df.head()

,num_processo,data_ajuizamento,data_primeira_movimentacao_processual,data_ultima_movimentacao,autor,reu,tutela_antecipada,vara,titular_substituto,classe_atual,...,resultado_1_grau,conteudo_sentenca,n_dias_ate_sentenca,n_tokens_sentenca,relator_acordao,data_sessao_tr,data_acordao,conteudo_acordao,resultado_2_grau,n_tokens_acordao
0,0500454-62.2017.4.05.8402,2017-03-06 00:00:00,2017-03-06 11:01:00,2017-10-30 10:07:00,Lourival Francisco da Silva (027.173.384-54)\n...,Fazenda Nacional (00.394.460/0239-40)\nMunicíp...,N,9-RN,Titular,Execução de Título Judicial,...,PROCEDENTE,SENTENÇA 1. Relatório Cuida-se de ação propost...,76.307593,1309,Almiro José da Rocha Lemos,2017-08-09 09:00:00,2017-08-14 10:46:34,PROCESSO 0500454-62.2017.4.05.8402 EMENTA:DIR...,NÃO PROVIMENTO,381.0
1,0517700-16.2013.4.05.8013,2013-10-09 00:00:00,2013-10-09 20:54:00,NaN,MARIA LUCIA CARDOSO (382.242.084-00)\nEmanuel ...,Caixa Econômica Federal (00.360.305/0001-04),N,9-AL,Titular,Procedimento do Juizado Especial Cível,...,EXTINTO SEM MÉRITO,"SENTENÇA Trata-se de ação especial cível, em c...",20.844491,263,NaN,NaN,NaN,NaN,NaN,0.1
2,0502388-07.2007.4.05.8305,2007-07-13 00:00:00,2007-07-13 13:51:00,NaN,QUITÉRIA BARBOSA DA SILVA NASCIMENTO (062.819....,Instituto Nacional do Seguro Social - INSS (29...,N,23-PE,Titular,Procedimento do Juizado Especial Cível,...,EXTINTO SEM MÉRITO,SENTENÇA Vistos etc. Trata-se de ação especial...,143.822245,956,NaN,NaN,NaN,NaN,NaN,0.1
3,0503551-02.2014.4.05.8103,2014-04-02 00:00:00,2014-04-02 11:02:00,NaN,Emanuela Carneiro Tabosa (777.839.723-49)\nBru...,UNIÃO (FAZENDA NACIONAL) - Procuradoria da Faz...,N,19-CE,Titular,Procedimento do Juizado Especial Cível,...,PROCEDENTE,SENTENÇA I - RELATÓRIO Cuida-se de ação em que...,21.360567,3037,CÍNTIA MENEZES BRUNETTA,2014-06-25 13:30:00,2014-06-27 16:14:43,"VOTO Relatório dispensado, nos termos do art. ...",NÃO PROVIMENTO,1214.0
4,0503488-94.2016.4.05.8300,2016-03-21 00:00:00,2016-03-21 11:04:00,2019-02-19 13:16:00,Lúcio Flavio de Lima Albuquerque (095.403.004-...,ANGELA MARIA DO NASCIMENTO SILVA (157.771.544-...,N,19-PE,Substituto,Procedimento do Juizado Especial Cível,...,IMPROCEDENTE,SENTENÇA Tipo A I – RELATÓRIO Dispensado o rel...,784.066493,2099,Joaquim Lustosa Filho,2018-06-19 10:00:00,2018-06-19 15:01:56,EMENTA PREVIDENCIÁRIO. PENSÃO POR MORTE. PROVA...,NÃO PROVIMENTO,1962.0


In [ ]:
columns = ['assunto_cnj','conteudo_sentenca']
df = df[columns]

In [ ]:
print("Shape depois da limpeza: ", df.shape)

Shape depois da limpeza:  (5000, 2)


In [ ]:
print(df.dtypes.value_counts())

object    2
dtype: int64


In [ ]:
df['assunto_cnj'].value_counts()

Direito Previdenciário                                         4059
Direito Administrativo e outras matérias do Direito Público     471
Direito do Consumidor                                           171
Direito Civil                                                   146
Direito Tributário                                              129
Direito Processual Civil e do Trabalho                           12
Direito do Trabalho                                              11
Direito Penal                                                     1
Name: assunto_cnj, dtype: int64

Como o assunto *Direito Penal* possui apenas um dado, não é possível dividí-lo entre treinamento e teste. Assim, vamos retirar essa linha do dataset.

In [ ]:
df = df[df['assunto_cnj'] != 'Direito Penal']
df['assunto_cnj'].value_counts()

Direito Previdenciário                                         4059
Direito Administrativo e outras matérias do Direito Público     471
Direito do Consumidor                                           171
Direito Civil                                                   146
Direito Tributário                                              129
Direito Processual Civil e do Trabalho                           12
Direito do Trabalho                                              11
Name: assunto_cnj, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4999 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   assunto_cnj        4999 non-null   object
 1   conteudo_sentenca  4999 non-null   object
dtypes: object(2)
memory usage: 117.2+ KB


In [ ]:
df.head()

,assunto_cnj,conteudo_sentenca
0,Direito Tributário,SENTENÇA 1. Relatório Cuida-se de ação propost...
1,Direito Civil,"SENTENÇA Trata-se de ação especial cível, em c..."
2,Direito Previdenciário,SENTENÇA Vistos etc. Trata-se de ação especial...
3,Direito Tributário,SENTENÇA I - RELATÓRIO Cuida-se de ação em que...
4,Direito Previdenciário,SENTENÇA Tipo A I – RELATÓRIO Dispensado o rel...


In [ ]:
# Gerando um "clean data file"
df.to_csv(artifact_name,index=False)

In [ ]:
# Criação e configuração do novo artefato no wandb
artifact = wandb.Artifact(name=artifact_name,
                          type=artifact_type,
                          description=artifact_description)
artifact.add_file(artifact_name)

<ManifestEntry digest: jQV4LLQND79g2+DLxPAsWA==>

In [ ]:
# Upload do artifact no Wandb
run.log_artifact(artifact)

In [ ]:
# close the run
run.finish()